In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

A = np.array([
    [0,1,1,1],
    [1,0,0,0],
    [1,0,0,1],
    [1,0,1,0]
])
#A = np.array([
#    [0,1,1,0,0,0,0,0,0,0,0],
#    [1,0,1,0,0,0,0,0,0,0,0],
#    [1,1,0,1,0,0,0,0,0,1,0],
#    [0,0,1,0,1,1,1,0,0,0,0],
#    [0,0,0,1,0,1,0,0,0,0,0],
#    [0,0,0,1,1,0,1,0,0,0,0],
#    [0,0,0,1,0,1,0,0,0,0,0],
#    [0,0,0,0,0,0,0,0,1,1,1],
#    [0,0,0,0,0,0,0,1,0,1,1],
#    [0,0,1,0,0,0,0,1,1,0,1],
#    [0,0,0,0,0,0,0,1,1,1,0]
#])

#for karate club case
A = nx.to_numpy_array(nx.karate_club_graph())

communities = [frozenset(x for x in range(len(A[0])))]
cluster_count = 0

: 

The following block of code creates 4 dictionaries which stores the value of all the metrics with iteration number as a key and a list with metric value of all nodes as its value. As each time the function is called only when the cluster count is updated, I have used cluster_count to be the iteration number.

In [ ]:
degree_centrality = {}
betweenness_centrality = {}
closeness_centrality = {}
clustering_coefficient = {}

def calculate_parameters():
    degree_centrality[cluster_count] = []
    betweenness_centrality[cluster_count] = []
    closeness_centrality[cluster_count] = []
    clustering_coefficient[cluster_count] = []
    for i in range(A.shape[0]):
        degree_centrality[cluster_count].append(nx.degree_centrality(G)[i])
        betweenness_centrality[cluster_count].append(nx.betweenness_centrality(G)[i])
        closeness_centrality[cluster_count].append(nx.closeness_centrality(G)[i])
        clustering_coefficient[cluster_count].append(nx.clustering(G)[i])

The following code contains a function which allows us to add cluster into the list "communities" so that it can be used for marking each community in the graph.

Initially I make the B_c which is the modularity matrix for each new community created. Then, I create the sorted eigen value and eigen vector from which I get the eigen vector corresponding to the largest eigen vector. Then I create two empty list in which the node numbers will get added according to the community they belong.

In [ ]:
def clustering(cluster):
    global cluster_count
    B_c = np.zeros((len(cluster),len(cluster)))
    k=0
    m=0
    for i in cluster:
        for j in cluster:
            B_c[k][m] = B[i][j]
            m+=1
        k+=1
        m=0
    k =0
    eigenvalues, eigenvectors = np.linalg.eig(B_c)

    #for debugging
    print(cluster)
    print('Eigen values and vectors after sorting')
    print(eigenvalues,pd.DataFrame(eigenvectors))

    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    u1 = eigenvectors[:,0]

    #for debugging
    print(u1)
    print('Eigen values and vectors after sorting')
    print(eigenvalues,pd.DataFrame(eigenvectors))
    print("#----------------------------------------------------#")

    if eigenvalues[0]>0:
        print('what should we do')
        to_do_list.append([])
        to_do_list.append([])
        to_do_list.remove(cluster)
        for j in range(len(eigenvectors[0])):
            if u1[j]>0:
              to_do_list[-1].append(cluster[j])
            else:
              to_do_list[-2].append(cluster[j])
#------------------------------------------------------------------------#
  #If there is a case in which the eigen value is +ve and all the values in the
  #corresponding vectors are either +ve/-ve then it comes inside this if
  #statement
        if [] in to_do_list:
          print('It came over here')
          to_do_list.remove([])
          to_do_list.remove(cluster)
          communities.append(frozenset(cluster))
          cluster_count+=1
          return True
#------------------------------------------------------------------------#
        return False

    else:
       print('clustering updating')

       to_do_list.remove(cluster)
       communities.append(frozenset(cluster))
       cluster_count+=1
       return True

In [ ]:
def plot(G):
    color_map = {}
    for i, comm in enumerate(communities):
        for node in comm:
            color_map[node] = i
    for i in range(node_count):
        if i not in color_map.keys():
            color_map[i] = node_count
    colors = [color_map[node] for node in G.nodes()]
    pos = nx.spring_layout(G, seed=51)
    nx.draw(G, pos, with_labels=True, node_color=colors, cmap=plt.cm.tab10)
    plt.show()

In [ ]:
k = np.sum(A, axis=1)
edge_count = k.sum()/2
B = A - np.outer(k, k) / (2*edge_count)

G = nx.from_numpy_array(A)

node_count = A.shape[1]
to_do_list = [list(range(node_count))]
output_list = []

while to_do_list != []:
  for i in to_do_list:
    if clustering(i):
        nx.set_node_attributes(G, communities, "community")
        if to_do_list != []:
          print('Iteration Number:', cluster_count)
          plot(G)
        calculate_parameters()
        continue
    else:
        continue

In [ ]:
print(f'{cluster_count} clusters were identified\n')

from matplotlib.ticker import MaxNLocator

x_axis = list(range(1, cluster_count+1,1))
for i in range(node_count):
    degree_centrality_values = []
    betweenness_centrality_values = []
    closeness_centrality_values = []
    clustering_coefficient_values = []

    print('Centrality values of node',i)
    for j in degree_centrality.keys():
        degree_centrality_values.append(degree_centrality[j][i])
        betweenness_centrality_values.append(betweenness_centrality[j][i])
        closeness_centrality_values.append(closeness_centrality[j][i])
        clustering_coefficient_values.append(clustering_coefficient[j][i])

#Specifications for plotting
    ax = plt.gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    plt.plot(x_axis, degree_centrality_values, label='degree centrality')
    plt.plot(x_axis, betweenness_centrality_values, label='betweenness centrality')
    plt.plot(x_axis, closeness_centrality_values, label='closeness centrality')
    plt.plot(x_axis, clustering_coefficient_values, label='clustering coefficient')
    plt.title(f'Centrality values of node {i}')
    plt.xlabel('Iterations')
    plt.ylabel('Centrality Values')
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    plt.tight_layout()
    plt.show()